# Package

In [6]:
import os
import numpy as np
import pandas as pd
import pickle
import helper as hp


In [7]:
# state = 'ny'
state = 'ca'

data_dir = f'../Data_{state}/'
results_dir = f'../Results_{state}/'
fig_dir = f'../Figures_{state}/'


# Comstock

In [8]:
# Read comstock building characteristics data
com_building_state = pd.read_csv(
    f'../Data_{state}/com_building_{state}_meta.csv')

# Create a list of county IDs
county_id_df = com_building_state.loc[:, [
    'in.county', 'in.resstock_county_id']]
county_id_df = county_id_df.drop_duplicates()
county_id_list = county_id_df['in.county'].to_list()
county_id_list.sort()

# Get lists of unique hvac types
com_heating_type_list = list(
    pd.unique(com_building_state['in.hvac_heating_type']))
com_cooling_type_list = list(
    pd.unique(com_building_state['in.hvac_cooling_type']))


In [9]:
# Import ResStock data
resstock_amy2018_state_dir = os.path.join(
    data_dir, f'resstock_amy2018/{state}')

# Read correction factors for resstock in state
res_corr = pd.read_csv(os.path.join(
    resstock_amy2018_state_dir, f'correction_factors_{state}_2018.csv'))
res_corr = pd.concat([res_corr, res_corr.tail(1)], axis=0)
date_list = pd.date_range('2018-01-01', '2019-01-01', freq='1D')
res_corr = res_corr.set_index(date_list)
res_corr = res_corr.drop(
    columns=['Month', 'day_of_month', 'build_existing_model.state'])
res_corr = res_corr.resample('1H').ffill()
res_corr = res_corr.drop(res_corr.tail(1).index)


In [10]:
comstock_amy2018_bldg_dir = f'../Data_{state}/comstock_amy2018/building/'
comstock_amy2018_bldg_processed_dir = os.path.join(
    comstock_amy2018_bldg_dir, 'processed')
if not os.path.isdir(comstock_amy2018_bldg_processed_dir):
    os.makedirs(comstock_amy2018_bldg_processed_dir)

overwrite = False

for county_id in county_id_list:
    print('County ID', county_id)
    # If county's directory exists
    county_dir = os.path.join(comstock_amy2018_bldg_dir, county_id)
    if os.path.isdir(county_dir):

        # If processed pickle file doesn't already exist, process the data
        filename = os.path.join(
            comstock_amy2018_bldg_processed_dir, f'{county_id}_heat.pkl')
        if not os.path.isfile(filename) or overwrite:
            # Loop through hvac heating types
            com_bldg_heating_tot_dict = dict()
            for heating_type in com_heating_type_list:
                cond = (com_building_state['in.county'] == county_id) & (
                    com_building_state['in.hvac_heating_type'] == heating_type)
                com_bldg_id_sublist = list(com_building_state[cond]['bldg_id'])
                com_bldg_wt_sublist = list(com_building_state[cond]['weight'])
                print(heating_type, ':', len(com_bldg_id_sublist))
                if len(com_bldg_id_sublist) > 0:
                    com_bldg_heating_tot_dict[heating_type] = hp.calc_bldg_ts_tot(com_bldg_id_sublist, com_bldg_wt_sublist,
                                                                                  county_dir, res_corr)
                else:
                    com_bldg_heating_tot_dict[heating_type] = None
            # Write to pickle
            pickle.dump(com_bldg_heating_tot_dict, open(filename, 'wb'))
            print('Finished processing data and saved in:', filename)
        else:
            print('Processed file already exists:', filename)

        # If processed pickle file doesn't already exist, process the data
        filename = os.path.join(
            comstock_amy2018_bldg_processed_dir, f'{county_id}_cool.pkl')
        if not os.path.isfile(filename) or overwrite:
            # Loop through hvac cooling types
            com_bldg_cooling_tot_dict = dict()
            for cooling_type in com_cooling_type_list:
                cond = (com_building_state['in.county'] == county_id) & (
                    com_building_state['in.hvac_cooling_type'] == cooling_type)
                com_bldg_id_sublist = list(com_building_state[cond]['bldg_id'])
                com_bldg_wt_sublist = list(com_building_state[cond]['weight'])
                print(cooling_type, ':', len(com_bldg_id_sublist))
                if len(com_bldg_id_sublist) > 0:
                    com_bldg_cooling_tot_dict[cooling_type] = hp.calc_bldg_ts_tot(com_bldg_id_sublist, com_bldg_wt_sublist,
                                                                                  county_dir, res_corr)
                else:
                    com_bldg_cooling_tot_dict[cooling_type] = None
            # Write to pickle
            pickle.dump(com_bldg_cooling_tot_dict, open(filename, 'wb'))
            print('Finished processing data and saved in:', filename)
        else:
            print('Processed file already exists:', filename)


County ID G0600010
Processed file already exists: ../Data_ca/comstock_amy2018/building/processed\G0600010_heat.pkl
Processed file already exists: ../Data_ca/comstock_amy2018/building/processed\G0600010_cool.pkl
County ID G0600030
Processed file already exists: ../Data_ca/comstock_amy2018/building/processed\G0600030_heat.pkl
Processed file already exists: ../Data_ca/comstock_amy2018/building/processed\G0600030_cool.pkl
County ID G0600050
Processed file already exists: ../Data_ca/comstock_amy2018/building/processed\G0600050_heat.pkl
Processed file already exists: ../Data_ca/comstock_amy2018/building/processed\G0600050_cool.pkl
County ID G0600070
Processed file already exists: ../Data_ca/comstock_amy2018/building/processed\G0600070_heat.pkl
Processed file already exists: ../Data_ca/comstock_amy2018/building/processed\G0600070_cool.pkl
County ID G0600090
Processed file already exists: ../Data_ca/comstock_amy2018/building/processed\G0600090_heat.pkl
Processed file already exists: ../Data_ca